In [ ]:
# %pip install torchtext torchdata portalocker

In [10]:
import torch
from torch import nn
from torchtext.datasets import AG_NEWS
# the AG_NEWS dataset iterators yield the raw data as a tuple of label and text.

In [2]:
train_iter = iter(AG_NEWS(split="train"))

In [3]:
next(train_iter)

(3,
 "Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\\band of ultra-cynics, are seeing green again.")

In [4]:
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

tokenizer = get_tokenizer("basic_english")

In [5]:
def yield_tokens(data_iter):
    for _, text in data_iter:
        yield tokenizer(text)


vocab = build_vocab_from_iterator(yield_tokens(train_iter), specials=["<unk>"])
vocab.set_default_index(vocab["<unk>"])

In [6]:
# use the vocab to convert tokens to integers
vocab(tokenizer('here is an example'))

[475, 21, 30, 5297]

In [7]:
# text pipe to convert words to integers and labels to integers
text_pipeline = lambda x: vocab(tokenizer(x))  # Returns the indices of the words in the given sentence
label_pipeline = lambda x: int(x) - 1  # Returns the label index

In [8]:
print(text_pipeline("Lets take a different route"))
label_pipeline("7")

[2972, 189, 5, 1616, 3944]


6

In [14]:
test_tensor = torch.tensor(text_pipeline('Lets check tensor output.'), dtype=torch.int64)
test_tensor.size(0)

5

In [25]:
vocab(tokenizer('Lets'))  # Tokenizer does additional work on the words

[2972]

In [15]:
test_tensor

tensor([2972, 1651,    0, 1520,    1])

In [9]:
from torch.utils.data import DataLoader

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def collate_batch(batch):
    "Batch of labels and texts will processed and returned as label_list & text_list"
    label_list, text_list, offsets = [], [], [0]  # instantiate three lists
    for _label, _text in batch:  # will contain a list of text, labels
        label_list.append(label_pipeline(_label))  # append to label_list
        processed_text = torch.tensor(text_pipeline(_text), dtype=torch.int64)  # tensors are created with text_pipeline
        text_list.append(processed_text)  # Append it to text_list
        offsets.append(processed_text.size(0))  
    label_list = torch.tensor(label_list, dtype=torch.int64)
    offsets = torch.tensor(offsets[:-1]).cumsum(dim=0)  # cumulatively add all the offsets of each sentence
    text_list = torch.cat(text_list)  # concatenate all the text sentences
    return label_list.to(device), text_list.to(device), offsets.to(device)  # return the data


train_iter = AG_NEWS(split="train")
dataloader = DataLoader(
    train_iter, batch_size=8, shuffle=False, collate_fn=collate_batch
)

nn.EmbeddingBag with the default mode of “mean” computes the mean value of a “bag” of embeddings. Although the text entries here have different lengths, nn.EmbeddingBag module requires no padding here since the text lengths are saved in offsets.

In [21]:
embedbag = nn.EmbeddingBag(2973, 10, sparse=False)

In [22]:
# How embed bag works
embedbag(test_tensor, torch.tensor([0, 4]))

tensor([[-0.4847,  0.2648,  0.9111, -0.2060, -0.0313,  0.0329,  0.5419, -0.2985,
          0.9735, -0.6527],
        [-1.4191,  2.0908, -1.1260, -1.5694,  0.4500, -0.2920,  0.0853,  1.6927,
         -0.5384, -1.2486]], grad_fn=<EmbeddingBagBackward0>)

In [23]:
# an EmbeddingBag module containing 10 tensors of size 3
embedding_sum = nn.EmbeddingBag(10, 3, mode='sum')
# a batch of 2 samples of 4 indices each
input = torch.tensor([1, 2, 4, 5, 4, 3, 2, 9], dtype=torch.long)
offsets = torch.tensor([0, 4], dtype=torch.long)
embedding_sum(input, offsets)

tensor([[-0.8895,  1.6600, -3.3034],
        [-0.0200,  0.7511, -4.7512]], grad_fn=<EmbeddingBagBackward0>)

In [24]:
# Example with padding_idx
embedding_sum = nn.EmbeddingBag(10, 3, mode='sum', padding_idx=2)
input = torch.tensor([2, 2, 2, 2, 4, 3, 2, 9], dtype=torch.long)
offsets = torch.tensor([0, 4], dtype=torch.long)
embedding_sum(input, offsets)

tensor([[ 0.0000,  0.0000,  0.0000],
        [-1.6157, -1.5974,  0.1489]], grad_fn=<EmbeddingBagBackward0>)

In [25]:

# An EmbeddingBag can be loaded from an Embedding like so
embedding = nn.Embedding(10, 3, padding_idx=2)
embedding_sum = nn.EmbeddingBag.from_pretrained( embedding.weight,
        padding_idx=embedding.padding_idx,
        mode='sum')

In [26]:
from torch import nn


class TextClassificationModel(nn.Module):
    def __init__(self, vocab_size, embed_dim, num_class):
        super(TextClassificationModel, self).__init__()
        self.embedding = nn.EmbeddingBag(vocab_size, embed_dim, sparse=False)  # intialize embedding bag obj
        self.fc = nn.Linear(embed_dim, num_class)  # create Linear model
        self.init_weights()  # init_weights method is defined below

    def init_weights(self):
        # assigning weights uniformly
        initrange = 0.5
        self.embedding.weight.data.uniform_(-initrange, initrange)
        self.fc.weight.data.uniform_(-initrange, initrange)
        self.fc.bias.data.zero_()

    def forward(self, text, offsets):
        # model embeds the text
        embedded = self.embedding(text, offsets)
        return self.fc(embedded)

1 : World

2 : Sports

3 : Business

4 : Sci/Tec

In [27]:
train_iter = AG_NEWS(split="train")
# getting the number of classes using set
num_class = len(set([label for (label, text) in train_iter]))  # 4
vocab_size = len(vocab)  # 95418
emsize = 64  # embed size
model = TextClassificationModel(vocab_size, emsize, num_class).to(device)

In [30]:
from torch.utils.data.dataset import random_split
from torchtext.data.functional import to_map_style_dataset

# Hyperparameters
EPOCHS = 10  # epoch
LR = 5  # learning rate
BATCH_SIZE = 64  # batch size for training

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=LR)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.1)
total_accu = None

train_iter, test_iter = AG_NEWS()
# Convert iterable-style dataset to map-style dataset.
train_dataset = to_map_style_dataset(train_iter)
test_dataset = to_map_style_dataset(test_iter)

num_train = int(len(train_dataset) * 0.95)

In [32]:
num_train

114000

In [35]:
import time


def train(dataloader):
    model.train()
    total_acc, total_count = 0, 0
    log_interval = 500  # logging interval
    start_time = time.time()
    # starting the for loop for training
    for idx, (label, text, offsets) in enumerate(dataloader):
        optimizer.zero_grad()
        predicted_label = model(text, offsets)  # based on forward pass
        loss = criterion(predicted_label, label)  # getting the loss
        loss.backward()  # back propagation
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.1)
        # torch.nn.utils.clip_grad_norm_ performs gradient clipping. It is used to mitigate the problem of exploding gradients, which is of particular concern for recurrent networks (which LSTMs are a type of).
        optimizer.step()
        total_acc += (predicted_label.argmax(1) == label).sum().item()
        total_count += label.size(0)
        if idx % log_interval == 0 and idx > 0:
            elapsed = time.time() - start_time
            print(
                "| epoch {:3d} | {:5d}/{:5d} batches "
                "| accuracy {:8.3f}".format(
                    epoch, idx, len(dataloader), total_acc / total_count
                )
            )
            total_acc, total_count = 0, 0
            start_time = time.time()


def evaluate(dataloader):
    model.eval()
    total_acc, total_count = 0, 0

    with torch.no_grad():
        for idx, (label, text, offsets) in enumerate(dataloader):
            predicted_label = model(text, offsets)  # predict without calculating grad
            loss = criterion(predicted_label, label)  # check the loss
            total_acc += (predicted_label.argmax(1) == label).sum().item()  # get accuracy
            total_count += label.size(0)  # count the total
    return total_acc / total_count

In [34]:
split_train_, split_valid_ = random_split(
    train_dataset, [num_train, len(train_dataset) - num_train]
)  # splitting the train_dataset into train and validation dataset

train_dataloader = DataLoader(
    split_train_, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_batch
)
valid_dataloader = DataLoader(
    split_valid_, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_batch
)
# test dataset is left alone
test_dataloader = DataLoader(
    test_dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_batch
)

In [ ]:
for epoch in range(1, EPOCHS + 1):
    epoch_start_time = time.time()
    train(train_dataloader)
    accu_val = evaluate(valid_dataloader)
    if total_accu is not None and total_accu > accu_val:
        scheduler.step()
    else:
        total_accu = accu_val
    print("-" * 59)
    print(
        "| end of epoch {:3d} | time: {:5.2f}s | "
        "valid accuracy {:8.3f} ".format(
            epoch, time.time() - epoch_start_time, accu_val
        )
    )
    print("-" * 59)

In [38]:
print("Checking the results of test dataset.")
accu_test = evaluate(test_dataloader)
print("test accuracy {:8.3f}".format(accu_test))

Checking the results of test dataset.
test accuracy    0.910


In [41]:
ag_news_label = {1: "World", 2: "Sports", 3: "Business", 4: "Sci/Tec"}


def predict(text, text_pipeline):
    with torch.no_grad():
        text = torch.tensor(text_pipeline(text)).to(device)
        output = model(text, torch.tensor([0]).to(device))
        return output.argmax(1).item() + 1


ex_text_str = "MEMPHIS, Tenn. – Four days ago, Jon Rahm was \
    enduring the season’s worst weather conditions on Sunday at The \
    Open on his way to a closing 75 at Royal Portrush, which \
    considering the wind and the rain was a respectable showing. \
    Thursday’s first round at the WGC-FedEx St. Jude Invitational \
    was another story. With temperatures in the mid-80s and hardly any \
    wind, the Spaniard was 13 strokes better in a flawless round. \
    Thanks to his best putting performance on the PGA Tour, Rahm \
    finished with an 8-under 62 for a three-stroke lead, which \
    was even more impressive considering he’d never played the \
    front nine at TPC Southwind."

model = model.to("cuda")

print("This is a %s news" % ag_news_label[predict(ex_text_str, text_pipeline)])

This is a Sports news


In [42]:
model_path = 'torchtext_classification.pth'
torch.save(model, model_path)

In [43]:
chekpoint = {
    'epoch':10,
    'model': model.state_dict(),
    'optimiser': optimizer.state_dict()
}
torch.save(chekpoint, model_path)